<div class="licence">
<span>Licence CC BY-NC-ND</span>
<span>Thierry Parmentelat &amp; Arnaud Legout</span>
</div>

In [ ]:
from plan import plan; plan("avancés", "méthodes statiques")

# méthodes statiques et de classe

# à quoi ça sert

* les méthodes statiques et de classe
  * peuvent travailler sur les arguments d’une classe
  * sans avoir besoin d’une instance
* par exemple, pour compter le nombre d’instances d’une classe
* ou pour fournir des utilitaires qui créent des instances

# méthodes *unbound* et *bound*

* une méthode *unbound* est une méthode appelée sur la classe
  * l’instance n’est pas automatiquement passée comme premier argument
  * c’est un objet fonction classique qui n’a pas besoin d’avoir une instance comme premier argument

In [ ]:
class C:
    def f(self):
        print("demo", self)
C.f

# méthodes *unbound* et *bound*

In [ ]:
C.f(1)   # on peut passer n’importe quel objet

In [ ]:
i = C()
C.f(i)   # on peut évidemment passer une instance

# méthodes *unbound* et *bound*

* une méthode *bound* est une méthode appelée sur l’instance
  * l’instance est automatiquement passée comme premier argument de la méthode
  * c’est un objet *bound method*

In [ ]:
class C:
     def f(self):
         print(self)
i = C()
i.f     # équivalent à C.f(i)

# comment appeler une méthode sans instance ?
(d’une classe ou d’un instance)

* une méthode appelée sur une instance est *bound*, elle prend automatiquement comme premier argument l’instance (`self`)
* par contre, une méthode appelée sur une classe est une fonction classique
* comment appeler une méthode qui travaille sur les arguments de la classe indifféremment d’une classe ou d’une instance
  * par exemple pour compter le nombre d’instances

# cas 1 : méthode sans argument

In [ ]:
class C:
    
    numInstances = 0

    def __init__(self):
        C.numInstances += 1
        
    # méthode qui ne prend
    # pas self en argument
    def printNumInstances(): 
        print("Nombre d'instances : {}"
              .format((C.numInstances)))

In [ ]:
# je peux envoyer la méthode
# à la classe elle-même
C.printNumInstances() 

In [ ]:
# mais pas sur une instance
i = C()

try:     
    i.printNumInstances()
except TypeError as e:
    print("OOPS", e)

# cas 2 : méthode avec `self` comme argument

In [ ]:
class C:

    numInstances = 0                         

    def __init__(self):
        C.numInstances += 1

    # méthode habituelle avec self
    def printNumInstances(self): 
        print("Nombre d'instances : {}"
              .format((C.numInstances)))

In [ ]:
# cette fois-ci c'est l'inverse, 
# je peux envoyer 
# la méthode à une instance
i = C()
i.printNumInstances() 

In [ ]:
# mais pas sur la classe
try:
    C.printNumInstances()
except TypeError as e:
    print("OOPS", e)    

# en résumé

* on ne peut pas appeler uniformément la même méthode depuis la classe et l’instance 
  * soit on peut l’appeler de l’instance, mais pas de la classe
  * soit on peut l’appeler de la classe, mais pas de l’instance

# utiliser une fonction

* la solution naïve à ce problème
  * sortir la méthode de la classe
  * en faire en quelque sorte une "une méthode de module"
  * mais ça n'est vraiment pas très élégant
  * notamment ça casse l'encapsulation

# méthode de module

In [ ]:
def printNumInstances(): # méthode de module
    print("Nombre d'instances : {} "
          .format((C.numInstances)))

class C:
    numInstances = 0
    def __init__(self):
        C.numInstances = C.numInstances + 1

In [ ]:
printNumInstances()

In [ ]:
a = C()
printNumInstances()

# problème avec les méthodes de module

* le code travaillant sur la classe n’est pas lié à la classe
  * maintenance difficile
  * lecture du code difficile
* pas de possibilité de customisation par héritage

# méthodes statiques et de classe

* pour appeler une méthode sans instance (d’une classe ou d’une instance), il y a deux possibilités
* les méthodes statiques ne prennent pas l’instance en premier argument
  * indépendante de l’instance
  * créées avec `staticmethod`
* les méthodes de classe prennent comme premier argument une classe (et non une instance)
  * indépendante de l’instance
  * créées avec `classmethod`

# méthodes statiques

In [ ]:
class C:

    numInstances = 0                         

    def __init__(self):
        C.numInstances += 1

    def printNumInstances():
        print(f"Nombre d'instances : {C.numInstances}")
    
    # en pratique ici on utilise
    # un décorateur; mais on n'en
    # a pas encore parlé ...
    printNumInstances = staticmethod(
        printNumInstances)

In [ ]:
# sur la classe
C.printNumInstances()

In [ ]:
# ou sur l'instance
i = C()
i.printNumInstances()

In [ ]:
C().printNumInstances()

### digression : décorateur

pour anticiper un peu, en pratique on remplace

```python
def printNumInstances():
    # blabla

printNumInstances = staticmethod(printNumInstances)
```

par plus simplement

```python
@staticmethod
def printNumInstances():
    # blabla
```


# méthodes statiques

* une méthode statique surchargée dans une sous classe doit être redéfinie comme statique dans la sous classe

In [ ]:
class SousC(C):
    
    # on redéfinit la méthode dans la sous-classe
    def printNumInstances():
        print("depuis sousC")
        C.printNumInstances()

    # il faudrait refaire la déclaration magique
    # comme staticmethod, car voici ce qui se passe 
    # si on ne le fait pas
    #printNumInstances = staticmethod(
    #    printNumInstances)

# méthodes statiques

In [ ]:
# avec une sous-classe qui ne redéclare pas 
# sa méthode comme staticmethod
# on se retrouve avec le problème initial
i = SousC()

In [ ]:
# et du coup on ne peut pas
# appeler la méthode sur une instance
try:     
    i.printNumInstances()
except TypeError as e:
    print("OOPS", e)

In [ ]:
# ça marche avec les classes par contre
SousC.printNumInstances()

In [ ]:
C.printNumInstances()

# méthodes statiques

In [ ]:
# si au contraire je redéclare la méthode 
# de la sous-classe comme statique
# en utilisant cette fois-ci un décorateur

class SousC(C):

    @staticmethod
    def printNumInstances():
        print("depuis SousC")
        C.printNumInstances()


In [ ]:
# maintenant ça marche 
# avec instances et classes
i = SousC()            
i.printNumInstances()  

In [ ]:
SousC.printNumInstances()

# méthodes statiques

In [ ]:
# une autre sous-classe de C 
# qui ne redéfinit pas la méthode
class AutreSousC(C):
    pass

In [ ]:
AutreSousC.printNumInstances()

In [ ]:
j = AutreSousC()
j.printNumInstances()

* `AutreSousC()` appelle automatiquement le constructeur de la classe `C`, ce qui incrémente le compteur d’instances

# méthodes de classe

In [ ]:
# imaginons maintenant qu'on veuille toujours
# compter les instances mais classe par classe
class C:

    numInstances = 0                         

    def __init__(self):
        C.numInstances += 1

    @classmethod
    def printNumInstances(cls):
        print(f"nb. instances de {cls.__name__}="
              f"{cls.numInstances}")

In [ ]:
c = C()

# sur l'instance
c.printNumInstances()

In [ ]:
# sur la classe
C.printNumInstances()

# méthode de classe

* avec une méthode de classe, le premier paramètre 
  * correspond à **la classe de** l'objet sujet de la méthode
  * du coup la convention est de l'appeler `cls` et non pas `self`

In [ ]:
class SousC(C):
    pass

In [ ]:
# on crée un objet C et un SousC
# -> 2 instances de plus comptabilisées
c, sousC = C(), SousC()
C.printNumInstances()

In [ ]:
SousC.printNumInstances()

In [ ]:
sousC.printNumInstances()

# nombre d’instances par sous classe

In [ ]:
class C:
    
    numInstances= 0

    def __init__(self):
        self.count()

    @classmethod
    def count(cls):
        print(f'incrementing {cls.__name__}')
        cls.numInstances += 1

    @classmethod
    def printNumInstances(cls):
        print(f"nb. instances de {cls.__name__}="
              f"{cls.numInstances}")

        
class SousC(C):
    numInstances= 0


In [ ]:
c = C()

In [ ]:
sous1, sous2 = SousC(), SousC()

In [ ]:
c.printNumInstances()
sous1.printNumInstances()
sous2.printNumInstances()

# quand utiliser `staticmethod` ?

* la méthode statique est adaptée 
  * lorsque l’on n'a pas d'instance sous la main
  * e.g. une usine à objets
* comme une fonction de module
  * mais dans l'espace de nom de la classe

# quand utiliser  `classmethod` ?

* la méthode de classe (puisqu’elle reçoit la classe lors de l’appel) est adaptée si 
  * on a un comportement spécifique en fonction de la sous classe
  * on veut travailler sur des attributs de la classe, mais on ne veut pas coder en dur son nom
* voir [cet exemple intéressant sur SO](http://stackoverflow.com/questions/12179271/python-classmethod-and-staticmethod-for-beginner) d’utilisation de `staticmethod` et `classmethod`